<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_05_regresion_y_clasificacion_scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 05 — Regresión y Clasificación con scikit‑learn


# Comparativo: Regresión Lineal vs Regresión Logística

# 📘 Regresión Lineal

## Concepto

* Modelo estadístico que **predice un valor numérico continuo** en función de variables independientes.
* Supone una **relación lineal** entre variables:

$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_n X_n + \epsilon
$$

* Donde:

  * $Y$ = variable dependiente (lo que queremos predecir).
  * $X_i$ = variables independientes (predictoras).
  * $\beta_i$ = coeficientes (pendientes).
  * $\epsilon$ = error.

---

## Ejemplo

* Predecir el **precio de una casa** en función de su superficie.

$$
Precio = \beta_0 + \beta_1 \cdot Superficie
$$

Si $\beta_0 = 50,000$ y $\beta_1 = 1,000$,
una casa de 100 m² tendría precio ≈ 150,000.

---

## Animación
https://phet.colorado.edu/es/simulations/least-squares-regression

---

## Gráfico Intuitivo

Una **línea recta** que mejor ajusta los puntos de datos.
La idea es minimizar el error cuadrático medio (**MSE**).

---

## Aplicaciones

* Predicción de ingresos en función de años de educación.
* Forecast de ventas según inversión en marketing.
* Estimar el riesgo crediticio (como variable continua, ej. probabilidad).

---

In [ ]:
# Ejemplo en Python

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X, y = make_regression(n_samples=300, n_features=1, noise=15.0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

lin_model = LinearRegression().fit(X_train, y_train)
y_pred = lin_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

plt.scatter(X_test, y_test, alpha=0.5)
order = np.argsort(X_test[:, 0])
plt.plot(X_test[order], y_pred[order], color="red")
plt.title(f"Regresión Lineal: RMSE={rmse:.2f}, R²={r2:.3f}")
plt.xlabel("X"); plt.ylabel("y")
plt.show()


In [ ]:
# --- Gráfico Predicción vs Realidad ---
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         'r--', lw=2)  # línea ideal
plt.xlabel("Valor Real (y_test)")
plt.ylabel("Predicción (y_pred)")
plt.title("Predicción vs Realidad")
plt.show()

# 📘 Regresión Logística

## Concepto

* Variante que sirve para **clasificación binaria** (Sí/No, 0/1).
* En lugar de predecir valores continuos, predice **probabilidades**:

$$
P(Y=1|X) = \frac{1}{1 + e^{-(\beta_0 + \beta_1X_1 + \dots + \beta_nX_n)}}
$$

* La función **sigmoide** transforma cualquier valor en un número entre 0 y 1.

---

## Ejemplo

* Predecir si un cliente **hará churn** (1) o no (0).

$$
Logit(P) = \ln\left(\frac{P}{1-P}\right) = \beta_0 + \beta_1 X
$$

Si el modelo predice $P=0.8$, interpretamos: **80% de probabilidad de churn**.

---

## Gráfico Intuitivo

* La curva de la regresión logística es en forma de **S (sigmoide)**.
* Para valores bajos de X → P cercano a 0.
* Para valores altos de X → P cercano a 1.
* Punto de corte (threshold, usualmente 0.5) → decide clase 0 o 1.

---

## Aplicaciones

* Detección de fraude (fraude = 1, no fraude = 0).
* Diagnóstico médico (enfermo = 1, sano = 0).
* Predicción de abandono (churn).

---

In [ ]:
# Ejemplo en Python

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, RocCurveDisplay

# Dataset binario 1D (válido)
Xc, yc = make_classification(
    n_samples=10000,
    n_features=1,
    n_redundant=0,
    n_informative=1,
    n_classes=2,
    n_clusters_per_class=1,
    class_sep=0.8,      # ↓ separa menos las clases
    flip_y=0.2,         # 20% de etiquetas ruidosas
    random_state=42
)

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    Xc, yc, test_size=0.25, stratify=yc, random_state=42
)

# Modelo
log_model = LogisticRegression().fit(Xc_train, yc_train)

y_pred = log_model.predict(Xc_test)
y_proba = log_model.predict_proba(Xc_test)[:, 1]

print("Accuracy:", accuracy_score(yc_test, y_pred))
print("F1:", f1_score(yc_test, y_pred))
print("AUC:", roc_auc_score(yc_test, y_proba))

In [ ]:
# Curva sigmoide
xs = np.linspace(Xc_test.min(), Xc_test.max(), 200).reshape(-1, 1)
sig = log_model.predict_proba(xs)[:, 1]

plt.scatter(Xc_test, yc_test, alpha=0.5)
plt.plot(xs, sig)
plt.title("Regresión Logística: Probabilidad P(Y=1|X)")
plt.xlabel("X"); plt.ylabel("Probabilidad")
plt.show()

In [ ]:
# Curva ROC
RocCurveDisplay.from_predictions(yc_test, y_proba)
plt.title("Curva ROC")
plt.show()

In [ ]:
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    PrecisionRecallDisplay, average_precision_score
)

# --- Reporte de clasificación ---
print("\n=== Classification Report ===")
print(classification_report(yc_test, y_pred, digits=3))

# --- Matriz de confusión (texto + plot) ---
cm = confusion_matrix(yc_test, y_pred)
print("\n=== Confusion Matrix ===\n", cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Matriz de Confusión")
plt.show()

# --- Curva Precision-Recall + Average Precision ---
ap = average_precision_score(yc_test, y_proba)
PrecisionRecallDisplay.from_predictions(yc_test, y_proba)
plt.title(f"Curva Precision-Recall (AP = {ap:.3f})")
plt.show()


# 📊 Diferencias Clave

| Aspecto         | Regresión Lineal      | Regresión Logística        |
| --------------- | --------------------- | -------------------------- |
| Tipo de salida  | Variable continua (ℝ) | Probabilidad (0 a 1)       |
| Función         | Recta                 | Sigmoide (S)               |
| Problema típico | Predicción de valores | Clasificación binaria      |
| Ejemplo         | Precio de casas       | ¿El cliente abandona o no? |
| Métrica común   | MSE, R²               | AUC, F1, LogLoss           |

---

👉 Una buena forma de recordarlo:

* **Lineal = línea recta = predecir números**.
* **Logística = lógica binaria = predecir sí/no con probabilidades**.

# Introducción al Machine Learning con Scikit-learn

## Objetivos
- Comprender qué es aprendizaje supervisado.
- Diferenciar entre regresión y clasificación.
- Usar Scikit-learn para entrenar un primer modelo.

---

## 1. Aprendizaje Supervisado
- **Regresión**: predice valores numéricos (ej: precio de una casa).  
- **Clasificación**: predice categorías (ej: churn = sí/no).  

Pipeline general:
1. Preparar datos.  
2. Dividir en train/test.  
3. Entrenar modelo.  
4. Evaluar resultados.  

---

## 2. Dataset de Ejemplo: Boston Housing (Regresión)

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

data = fetch_california_housing(as_frame=True)
df = data.frame
df.head()

---

## 3. Dataset de Ejemplo: Telco Churn (Clasificación)
Este código es un **preprocesamiento básico para un modelo de machine learning** sobre el dataset de *Telco Customer Churn* (abandono de clientes).

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

url = "https://raw.githubusercontent.com/Geo-y20/Telco-Customer-Churn-/refs/heads/main/Telco%20Customer%20Churn.csv"
df = pd.read_csv(url, index_col=0)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df["TotalCharges"] = df["TotalCharges"].fillna(df["MonthlyCharges"])
df.head()

In [ ]:
# One-Hot Encoding con pandas
df = pd.get_dummies(df, drop_first=True)

In [ ]:
df.dtypes

In [ ]:
# Separar variables y target
X = df.drop("Churn_Yes", axis=1)  # porque get_dummies crea Churn_No y Churn_Yes
y = df["Churn_Yes"]

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### 3.1. Importación de librerías

```python
import pandas as pd
from sklearn.model_selection import train_test_split
```

* **pandas**: para manipular datos en forma de tablas (dataframes).
* **train\_test\_split**: función de Scikit-learn para dividir los datos en conjuntos de entrenamiento y prueba.

---

### 3.2. Cargar el dataset

```python
url = "https://raw.githubusercontent.com/Geo-y20/Telco-Customer-Churn-/refs/heads/main/Telco%20Customer%20Churn.csv"
df = pd.read_csv(url, index_col=0)
```

* Se descarga un CSV desde GitHub.
* `index_col=0` indica que la primera columna del CSV debe usarse como índice del dataframe.

---

### 3.3. One-Hot Encoding (variables categóricas a numéricas)

```python
df = pd.get_dummies(df, drop_first=True)
```

* `pd.get_dummies()` convierte las variables categóricas en variables numéricas binarias (0/1).
* `drop_first=True` elimina una de las categorías para evitar la **trampa de la multicolinealidad** (cuando una categoría es redundante).

Ejemplo: si hay una columna `Gender` con valores `Male` y `Female`, se creará solo `Gender_Male` (0 o 1), y se elimina `Gender_Female` porque es redundante.

---

### 3.4. Separar variables predictoras (X) y la variable objetivo (y)

```python
X = df.drop("Churn_Yes", axis=1)  
y = df["Churn_Yes"]
```

* `Churn` es la variable objetivo (si el cliente se dio de baja o no).
* Como `get_dummies` crea `Churn_No` y `Churn_Yes`, se toma `Churn_Yes` como target.
* `X` son todas las demás columnas (variables explicativas).

---

### 3.5. Dividir en entrenamiento y prueba

```python
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
```

* Divide los datos en:

  * **70%** entrenamiento (`X_train`, `y_train`)
  * **30%** prueba (`X_test`, `y_test`)
* `random_state=42` asegura que la división sea **reproducible** (si corres el código varias veces, el resultado será el mismo).

---

### 3.6. Mostrar las dimensiones

```python
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
```

* Muestra el número de **filas (observaciones)** y **columnas (features)** en cada conjunto.
* Sirve para verificar que la división se hizo correctamente.

---

✅ **Resumen**:
Este script descarga un dataset de clientes, transforma las variables categóricas en numéricas, separa la variable objetivo (*churn*), divide los datos en entrenamiento y prueba, y finalmente imprime las dimensiones resultantes.

---

## 4. Ejemplo: Clasificación (Logistic Regression)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir y entrenar modelo
clf = LogisticRegression(max_iter=5000)
clf.fit(X_train, y_train)

# Predicciones
y_pred = clf.predict(X_test)

# Reporte de métricas
print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", cm)

# Visualizar con mapa de calor
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=clf.classes_,
            yticklabels=clf.classes_)
plt.xlabel("Predicciones")
plt.ylabel("Valores reales")
plt.title("Matriz de Confusión")
plt.show()


### 📌 Recordatorio de la fórmula

1. **Logit (score lineal):**

$$
z = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_n x_n
$$

2. **Sigmoide:**

$$
p = \sigma(z) = \frac{1}{1 + e^{-z}}
$$

---


In [ ]:
# --- Coeficientes, intercepto, logit y probabilidad

import numpy as np

# 1) Extraer parámetros del modelo (asumiendo clf = LogisticRegression ya entrenado)
beta  = np.asarray(clf.coef_, dtype=float).ravel()   # (n_features,)
beta0 = float(np.asarray(clf.intercept_, dtype=float).ravel()[0])

print("Intercepto (beta0):", beta0)
print("Coeficientes (beta):", beta)

# 2) Nombres de features si X_test es DataFrame; si no, nombres genéricos
try:
    feature_names = list(X_test.columns)
except AttributeError:
    # Si es ndarray/sparse, construimos nombres genéricos
    n_features = X_test.shape[1]
    feature_names = [f"x{i}" for i in range(n_features)]

# 3) Asegurar que X_test sea numérico ndarray (maneja DataFrame/ndarray)
if hasattr(X_test, "values"):   # DataFrame/Series de pandas
    X_mat = X_test.values
else:
    X_mat = X_test

# Si X_mat es sparse, conviértelo a denso de forma segura (o usa operaciones sparse si prefieres)
try:
    import scipy.sparse as sp
    if sp.issparse(X_mat):
        X_mat = X_mat.toarray()
except Exception:
    # Si no hay scipy o no es sparse, seguimos
    pass

X_mat = np.asarray(X_mat, dtype=float)

# 4) Cálculo del logit (score lineal) y probabilidad con estabilidad numérica
z = beta0 + X_mat @ beta

# Evitar overflow en exp: recorta z a [-709, 709] (aprox. límite de np.exp para float64)
z_clipped = np.clip(z, -709, 709)
p = 1.0 / (1.0 + np.exp(-z_clipped))  # sigmoide

# 5) Chequeo contra predict_proba de scikit-learn
p_sklearn = clf.predict_proba(X_test)[:, 1]
print("Max |p - p_sklearn|:", float(np.max(np.abs(p - p_sklearn))))

# 6) Impresión ordenada de coeficientes
print("\nCoeficientes por feature:")
for name, b in zip(feature_names, beta):
    print(f"{name:20s}: {b: .6f} ")
    #print(f"{name:20s}: {b: .6f}   (odds ratio: {np.exp(b):.6f})")

# 7) (Opcional) Mostrar algunos ejemplos de salida
print("\nPrimeras 5 probabilidades calculadas manualmente:", p[:5])
print("Primeras 5 probabilidades de predict_proba()   :", p_sklearn[:5])


In [ ]:
order = np.argsort(z)
plt.figure(figsize=(7,5))
plt.scatter(z, y_test, alpha=0.35, label="Observaciones (0/1)")
plt.plot(z[order], p[order], label="Sigmoide sobre z")
plt.xlabel("Logit (z = β0 + Xβ)")
plt.ylabel("P(Y=1|X)")
plt.title("Sigmoide respecto al logit (score lineal)")
plt.legend()
plt.show()

### Qué hace el código

```python
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir y entrenar modelo
clf = LogisticRegression(max_iter=5000)
clf.fit(X_train, y_train)

# Predicciones
y_pred = clf.predict(X_test)

# Reporte de métricas
print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", cm)

# Visualizar con mapa de calor
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=clf.classes_,
            yticklabels=clf.classes_)
plt.xlabel("Predicciones")
plt.ylabel("Valores reales")
plt.title("Matriz de Confusión")
plt.show()
```

* **`LogisticRegression(max_iter=5000)`**: clasificador para problemas binarios/multiclase. `max_iter` alto evita warnings de no convergencia.
* **`.fit(X_train, y_train)`**: entrena con el set de entrenamiento.
* **`.predict(X_test)`**: predice etiquetas en el set de prueba.
* **`classification_report`**: imprime *precision, recall, f1-score, support* por clase, más promedios (macro/weighted) y *accuracy* global.
* **`confusion_matrix`**: devuelve la tabla de aciertos/errores por clase.
* **`sns.heatmap(...)`**: dibuja la matriz; las etiquetas de ejes usan `clf.classes_` (en tu caso, `['False','True']`).

---

### Reporte

```
              precision    recall  f1-score   support
False             0.85      0.90      0.87      1539
True              0.68      0.57      0.62       574
accuracy                               0.81      2113
macro avg         0.77      0.74      0.75      2113
weighted avg      0.80      0.81      0.81      2113
```

* **Accuracy = 0.81**: el 81% de las predicciones totales fue correcto.
* **Clase `False` (no churn)**: muy buen *recall* (0.90) ⇒ el modelo identifica bien a quienes **se quedan**.
* **Clase `True` (churn)**: *recall* 0.57 ⇒ detecta **57%** de quienes se van; *precision* 0.68 ⇒ de los que predijo como churn, el **68%** realmente se va.
* **Macro avg**: promedio simple entre clases (útil si están desbalanceadas).
* **Weighted avg**: promedio ponderado por *support* (tamaño de cada clase).

### Matriz de confusión

```
[[1386  153]
 [ 245  329]]
```

* Filas = **valores reales** (`y_test`)
* Columnas = **predicciones** (`y_pred`)
* Orden: `False` primero, `True` después

Entonces:

* **TN (True Negative) = 1386** → Reales **False** (no churn) predichos **False**.
  Aciertos en “no se va”.
* **FP (False Positive) = 153** → Reales **False**, predichos **True**.
  “Falsa alarma”: el modelo dijo churn pero no se fueron.
  *Impacto*: podrías ofrecer incentivos innecesarios.
* **FN (False Negative) = 245** → Reales **True**, predichos **False**.
  “Se te escapan”: se fueron y no los detectaste.
  *Impacto*: pérdida de clientes sin acción preventiva (suele ser el error más costoso en churn).
* **TP (True Positive) = 329** → Reales **True**, predichos **True**.
  Aciertos detectando quienes se van.

### Métricas derivadas desde la matriz

Con TN=1386, FP=153, FN=245, TP=329 (total=2113):

* **Accuracy** = (TP+TN)/total ≈ **0.812** (≈ 0.81)
* **Precision (churn=True)** = TP/(TP+FP) ≈ **0.683**
* **Recall (churn=True)** = TP/(TP+FN) ≈ **0.573**
* **F1 (churn=True)** ≈ **0.623**

### ¿Qué puede hacerse para mejorar la detección de churn (reducir FN)?

* **Ajustar el umbral de decisión**: en lugar de `predict` (umbral 0.5), usa `predict_proba` y mueve el umbral para priorizar *recall* en la clase `True`.
  *Más recall* ⇒ menos FN (pero suben FP).
* **`class_weight='balanced'`** en la Regresión Logística si hay desbalance.
* **Estandarizar** las variables numéricas (`StandardScaler`) antes de entrenar.
* **Curvas ROC y Precision-Recall** para elegir el umbral óptimo según tu coste FP/FN.
* **Ingeniería de variables / modelos alternativos** (árboles/boosting) si procede.


---

## 5. Ejemplo: Regresión Lineal

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_california_housing
import pandas as pd

data = fetch_california_housing(as_frame=True)
df = data.frame

# Definir modelo
model = LinearRegression()
model.fit(data.data, data.target)

# Predicciones
preds = model.predict(data.data)

# Métricas
print("MSE:", mean_squared_error(data.target, preds))
print("R2:", r2_score(data.target, preds))

In [ ]:
# Intercepto y coeficientes
print("\nIntercepto (β0):", model.intercept_)
print("\nCoeficientes por variable:")
for name, coef in zip(data.feature_names, model.coef_):
    print(f"{name:15s}: {coef:.6f}")

### Recordatorio del modelo

El modelo ajustado es:

$$
\hat{y} = \beta_0 + \beta_1 \cdot x_1 + \beta_2 \cdot x_2 + \dots + \beta_p \cdot x_p
$$

donde:

* $\hat{y}$ = valor medio de la vivienda (en **cientos de miles de dólares**),
* $x_j$ = variables predictoras,
* $\beta_j$ = coeficiente de la variable $j$.

---

### Cómo leer los coeficientes

* **Intercepto (β₀)**: valor esperado de la variable objetivo cuando todas las features son 0. (No suele tener interpretación realista en datasets como este, porque “0 habitaciones” o “0 latitud” no son escenarios válidos).
* **Coeficientes (β)**: representan el **cambio promedio en el valor de la vivienda (en 100k USD)** por un incremento de 1 unidad en la variable, manteniendo las demás constantes.

Ejemplo con las variables más relevantes del dataset:

| Variable                                       | Interpretación del coeficiente (ejemplo con valores típicos)                                                                            |
| ---------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------- |
| **MedInc** (ingreso medio)                     | Si el ingreso medio en un bloque sube en 1 (es decir, 10.000 USD anuales), el precio medio de las casas aumenta en ≈ `β` × 100k USD.    |
| **HouseAge** (edad de la casa en años)         | Un año adicional en la antigüedad promedio de las casas se asocia con `β` × 100k USD en el precio, manteniendo todo lo demás constante. |
| **AveRooms** (habitaciones promedio por hogar) | Cada habitación adicional promedio está asociada con un cambio de `β` × 100k USD en el precio medio.                                    |
| **Population** (población en el bloque)        | Cada incremento de 1 persona se asocia con un cambio muy pequeño en el valor (`β` cercano a 0).                                         |
| **Latitude / Longitude**                       | Tienen coeficientes negativos grandes porque la ubicación (más al norte/sur/oeste/este) influye fuertemente en el valor.                |

---

### Ejemplo con valores concretos

Si el coeficiente de `MedInc` fuera ≈ **0.437**, significa que:

* Aumentar el ingreso medio en **10.000 USD** se asocia con un aumento de:

  $$
  0.437 \times 100{,}000 = 43{,}700 \, \text{USD}
  $$

  en el valor medio de la vivienda.

---

### 📊 Interpretación del modelo

### Intercepto (β₀)

* **-36.94** → el valor esperado de una vivienda sería negativo si todas las variables fueran 0.
  👉 Esto no tiene sentido práctico (ninguna casa tiene 0 habitaciones, ingreso = 0, latitud = 0, etc.).
  El intercepto solo asegura que el modelo “pase” cerca de los datos reales.

---

### Coeficientes

| Variable                                               | Coeficiente (β) | Interpretación práctica                                                                                                                                                                                                              |
| ------------------------------------------------------ | --------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **MedInc** (ingreso medio, en decenas de miles de USD) | **0.4367**      | Por cada **+10,000 USD** de ingreso medio en el distrito, el valor medio de las casas aumenta en **≈ 43,700 USD**.                                                                                                                   |
| **HouseAge** (edad promedio en años)                   | **0.0094**      | Cada **+1 año** de antigüedad promedio está asociado con **≈ 940 USD más** en el precio medio (un efecto pequeño).                                                                                                                   |
| **AveRooms** (habitaciones promedio por hogar)         | **-0.1073**     | Una **habitación adicional en promedio** se asocia con **≈ -10,732 USD** en el valor. ⚠️ Esto puede sonar raro, pero refleja multicolinealidad: distritos con muchas habitaciones por hogar tienden a ser más baratos en California. |
| **AveBedrms** (dormitorios promedio por hogar)         | **0.6451**      | Cada dormitorio adicional en promedio se asocia con **≈ 64,506 USD más**.                                                                                                                                                            |
| **Population** (número de personas en el distrito)     | **-0.000004**   | Cada persona adicional reduce el valor en apenas **≈ -0.40 USD** (efecto casi nulo).                                                                                                                                                 |
| **AveOccup** (personas promedio por hogar)             | **-0.0038**     | Cada persona extra por hogar está asociada con **≈ -378 USD** en el valor medio.                                                                                                                                                     |
| **Latitude** (latitud del distrito)                    | **-0.4213**     | Moverse 1 grado al norte (≈ 111 km) reduce el valor medio en **≈ -42,131 USD**.                                                                                                                                                      |
| **Longitude** (longitud del distrito)                  | **-0.4345**     | Moverse 1 grado al este (≈ 85 km en California) reduce el valor medio en **≈ -43,451 USD**.                                                                                                                                          |

---

## 🔑 Claves de interpretación

* **MedInc** es el predictor más fuerte y lógico: más ingreso ⇒ casas más caras.
* **Latitude / Longitude** capturan la ubicación: en California, estar más al norte o este (lejos de la costa sur/Los Ángeles/San Francisco) reduce el valor.
* **AveRooms** negativo y **AveBedrms** positivo reflejan correlaciones internas: distritos con muchas habitaciones suelen estar en zonas rurales/suburbanas más baratas.
* **Population** tiene un efecto prácticamente nulo.

---


### Qué hace el código

### 5.1. Importar librerías

```python
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
```

* **LinearRegression**: el modelo de regresión lineal de scikit-learn.
* **mean\_squared\_error**: función para calcular el **error cuadrático medio (MSE)**.
* **r2\_score**: función para calcular el **coeficiente de determinación (R²)**.

---

### 5.2. Definir y entrenar el modelo

```python
model = LinearRegression()
model.fit(data.data, data.target)
```

* Se crea un objeto `LinearRegression`.
* `.fit(X, y)` entrena el modelo:

  * `X = data.data` → las variables predictoras (características).
  * `y = data.target` → la variable objetivo (lo que queremos predecir).

El modelo busca la mejor línea (o hiperplano) que se ajuste a los datos.

---

### 5.3. Hacer predicciones

```python
preds = model.predict(data.data)
```

* Se usan las mismas variables (`data.data`) para generar predicciones.
* `preds` es un array con los valores estimados por el modelo para cada observación.

---

### 5.4. Evaluar el modelo con métricas

```python
print("MSE:", mean_squared_error(data.target, preds))
print("R2:", r2_score(data.target, preds))
```

* **MSE (Mean Squared Error)**:

  * Calcula el promedio de los errores al cuadrado:

    $$
    MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
    $$
  * Cuanto más bajo, mejor es el ajuste.

* **R² (Coeficiente de determinación)**:

  * Mide qué porcentaje de la variabilidad en `y` es explicado por el modelo.
  * Valores:

    * **1** → ajuste perfecto.
    * **0** → el modelo no explica nada.
    * **Negativo** → el modelo es peor que predecir la media.

---

✅ **Resumen:**
Este código entrena un modelo de **regresión lineal** con los datos (`data.data`, `data.target`), genera predicciones, y luego evalúa el modelo mostrando dos métricas:

* **MSE** (qué tan lejos están las predicciones de los valores reales).
* **R²** (qué tan bien explica el modelo la variación de los datos).


---

## 6. Preguntas de Discusión

1. ¿Qué diferencias hay entre regresión y clasificación?
2. ¿Qué tan importante es separar train/test?
3. ¿Por qué no debemos evaluar un modelo solo con accuracy?


### 1. ¿Qué diferencias hay entre **regresión** y **clasificación**?

* **Regresión**:

  * Predice un **valor continuo** (numérico).
  * Ejemplo: predecir el precio de una casa, la temperatura, las ventas mensuales.
  * Métricas comunes: MSE, RMSE, MAE, R².

* **Clasificación**:

  * Predice una **categoría** (discreta).
  * Puede ser binaria (sí/no, churn/no churn) o multiclase (tipo de flor, enfermedad A/B/C).
  * Métricas comunes: accuracy, precision, recall, F1, ROC AUC.

👉 Diferencia clave: **regresión = valores continuos**, **clasificación = clases o categorías**.

---

### 2. ¿Qué tan importante es separar **train/test**?

* **Fundamental** ✅.
* Si entrenas y evalúas en los mismos datos → el modelo se "aprende de memoria" (overfitting) y obtienes métricas artificialmente buenas.
* El **conjunto de entrenamiento (train)** se usa para ajustar los parámetros del modelo.
* El **conjunto de prueba (test)** simula datos nuevos nunca vistos, y te da una estimación real de cómo rendirá el modelo en producción.
* A veces también se usa un **validation set** o **cross-validation** para afinar hiperparámetros antes de probar en test.

---

### 3. ¿Por qué no debemos evaluar un modelo solo con **accuracy**?

* El accuracy mide:

  $$
  \text{Accuracy} = \frac{\text{Aciertos}}{\text{Total de ejemplos}}
  $$

  Es útil cuando las clases están balanceadas.

* Pero en problemas **desbalanceados** puede ser engañoso:

  * Ejemplo: dataset de churn con 90% clientes que se quedan y 10% que se van.
  * Si el modelo predice **siempre “se queda”**, tendrá **90% accuracy** pero **0% recall para churn** (nunca detecta a quienes se van).
  * En este caso, métricas como **precision, recall, F1-score, ROC-AUC** son más informativas.

👉 En tareas críticas (fraude, churn, cáncer), **recall y precision son más importantes que accuracy**.

---

✅ **Resumen**:

1. Regresión predice valores continuos, clasificación categorías.
2. Separar train/test evita sobreajuste y mide rendimiento real.
3. Accuracy puede ser engañoso, especialmente con clases desbalanceadas: hay que mirar métricas adicionales.


## 📊 Comparación: Regresión vs Clasificación

| Aspecto                             | **Regresión** (valores continuos)                                                                                              | **Clasificación** (clases/categorías)                                           |
| ----------------------------------- | ------------------------------------------------------------------------------------------------------------------------------ | ------------------------------------------------------------------------------- |
| **Ejemplo**                         | Predecir el precio de una casa, la temperatura, ventas mensuales                                                               | Predecir si un cliente hará churn, si un email es spam/no spam                  |
| **Variable objetivo (y)**           | Numérica continua (ej. 250.45, 32.7)                                                                                           | Discreta (ej. {0,1} o {Rojo, Azul, Verde})                                      |
| **Algoritmos típicos**              | Regresión lineal, regresión polinómica, SVR, Random Forest Regressor                                                           | Regresión logística, SVM, Árboles de decisión, Random Forest, Redes Neuronales  |
| **Métricas comunes**                | - MSE (Error Cuadrático Medio)<br>- RMSE (Raíz del MSE)<br>- MAE (Error Absoluto Medio)<br>- R² (Coeficiente de determinación) | - Accuracy<br>- Precision<br>- Recall (Sensibilidad)<br>- F1-score<br>- AUC-ROC |
| **Interpretación**                  | Cuán cerca están las predicciones de los valores reales (se mide error en unidades del target)                                 | Qué tan bien clasifica ejemplos en sus categorías correctas                     |
| **Riesgo al usar solo una métrica** | Usar solo R² puede ser engañoso si los datos no son lineales                                                                   | Usar solo Accuracy es peligroso si las clases están desbalanceadas              |

---

✅ Con esta tabla queda claro que:

* En **regresión** interesa **minimizar el error numérico**.
* En **clasificación** interesa **equilibrar precisión, recall y F1**, sobre todo en datasets desbalanceados.
